https://www.analyticsvidhya.com/blog/2020/10/create-image-classification-model-python-keras/

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import cv2

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential

import os
import glob as gb
from skimage.io import imread, imshow
from scipy import spatial
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Add, Dropout,Dense, Activation, ZeroPadding2D,\
               BatchNormalization, Flatten, Conv2D, GlobalAveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from tensorflow.keras.optimizers import SGD, Adam


In [2]:
trainpath= ('E:/4sd_t extension/python_code/brain/decomposition_brain_Training/')
testpath = ('E:/4sd_t extension/python_code/brain/ImgsTesting/')

In [3]:
labels=os.listdir(trainpath)
labels

['glioma_tumor_0',
 'glioma_tumor_1',
 'meningioma_tumor_0',
 'meningioma_tumor_1',
 'meningioma_tumor_2',
 'no_tumor_0',
 'no_tumor_1',
 'pituitary_tumor_0',
 'pituitary_tumor_1']

In [4]:
Folder_name=[]
folder_item_numbers = []
for folder in  os.listdir(trainpath ) : 
    files = gb.glob(pathname= str( trainpath  + folder + '/*.jpg'))
    Folder_name.append(folder)
    folder_item_numbers.append(len(files))
foldernames=pd.DataFrame({'Folder_name':Folder_name})
itemnumbers=pd.DataFrame({'Traning Image Numbers':folder_item_numbers})
informations=pd.concat([foldernames,itemnumbers],axis=1)
print(informations)

          Folder_name  Traning Image Numbers
0      glioma_tumor_0                    308
1      glioma_tumor_1                    518
2  meningioma_tumor_0                    272
3  meningioma_tumor_1                    278
4  meningioma_tumor_2                    272
5          no_tumor_0                    183
6          no_tumor_1                    212
7   pituitary_tumor_0                    320
8   pituitary_tumor_1                    507


In [5]:
image_class = {'glioma_tumor_0': 0,'glioma_tumor_1': 1, 'meningioma_tumor_0': 2, 'meningioma_tumor_1': 3, 'meningioma_tumor_2': 4,  'no_tumor_0': 5 ,\
                         'no_tumor_1': 6 , 'pituitary_tumor_0': 7,'pituitary_tumor_1': 8}
len(image_class)


9

In [6]:
#loading training data and resize it and collect it in one folder
new_size=224    
X_train = []
y_train = []
Folder_name=[]
folder_item_numbers = []

for folder in  os.listdir(trainpath ) : 
    Folder_name.append(folder)
    files = gb.glob(pathname= str( trainpath  + folder + '/*.jpg'))
    folder_item_numbers.append(len(files))
    for file in files: 
        image_class = {'glioma_tumor_0': 0,'glioma_tumor_1': 1, 'meningioma_tumor_0': 2, 'meningioma_tumor_1': 3, 'meningioma_tumor_2': 4,  'no_tumor_0': 5 ,\
                         'no_tumor_1': 6 , 'pituitary_tumor_0': 7,'pituitary_tumor_1': 8}
        orignal_image = cv2.imread(file)
        image = cv2.cvtColor(orignal_image, cv2.COLOR_BGR2RGB)
        resized_image = cv2.resize(image , (new_size,new_size))
        X_train.append(resized_image)
        y_train.append(image_class[folder])

foldernames=pd.DataFrame({'Folder_name':Folder_name})
itemnumbers=pd.DataFrame({'Traning Image Numbers':folder_item_numbers})
informations=pd.concat([foldernames,itemnumbers],axis=1)
print(informations)
print('--------------------------------------------------')        
#check items in X_test
print("items in X_train is:       ",len(X_train) , " items") 
print("items in y_train is:       ",len(y_train) , " items") 

          Folder_name  Traning Image Numbers
0      glioma_tumor_0                    308
1      glioma_tumor_1                    518
2  meningioma_tumor_0                    272
3  meningioma_tumor_1                    278
4  meningioma_tumor_2                    272
5          no_tumor_0                    183
6          no_tumor_1                    212
7   pituitary_tumor_0                    320
8   pituitary_tumor_1                    507
--------------------------------------------------
items in X_train is:        2870  items
items in y_train is:        2870  items


In [ ]:
#showing training images with labels
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_train),12))) : 
    plt.subplot(3,4,n+1)
    plt.imshow(X_train[i])   
    plt.axis('off')
    classes = {'glioma_tumor_0': 0,'glioma_tumor_1': 1, 'meningioma_tumor_0': 2, 'meningioma_tumor_1': 3, 'meningioma_tumor_2': 4,  'no_tumor_0': 5 ,\
                         'no_tumor_1': 6 , 'pituitary_tumor_0': 7,'pituitary_tumor_1': 8}
    def ImageClass(n):
        for x , y in classes.items():
            if n == y :
                return x
    plt.title(ImageClass(y_train[i]))

In [7]:
#shuffle data
from sklearn.utils import shuffle
img_size=224
X_train,y_train = shuffle(X_train, y_train)

x_train, x_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size= 0.8)   #test_size
print('Training Samples =:       ',len(x_train) ,'  and the number of labels is     ', len(y_train))
print('Validation Samples =:       ',len(x_valid) ,'  and the number of labels is     ', len(y_valid))


Training Samples =:        2296   and the number of labels is      2296
Validation Samples =:        574   and the number of labels is      574


In [8]:
#loading test data and resize it and collect it in one folder
new_size=224    
x_test = []
y_test = []
Folder_name=[]
folder_item_numbers = []

for folder in  os.listdir(testpath ) : 
    Folder_name.append(folder)
    files = gb.glob(pathname= str( testpath  + folder + '/*.jpg'))
    folder_item_numbers.append(len(files))
    for file in files: 
        image_class2 = {'glioma_tumor': 0,'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}
        orignal_image = cv2.imread(file)
        image = cv2.cvtColor(orignal_image, cv2.COLOR_BGR2RGB)
        resized_image = cv2.resize(image , (new_size,new_size))
        x_test.append(resized_image)
        y_test.append(image_class2[folder])

foldernames=pd.DataFrame({'Folder_name':Folder_name})
itemnumbers=pd.DataFrame({'Traning Image Numbers':folder_item_numbers})
informations=pd.concat([foldernames,itemnumbers],axis=1)
print(informations)
print('--------------------------------------------------')        
#check items in X_test
print("items in x_test is:       ",len(x_test) , " items") 
print("items in y_test is:       ",len(y_test) , " items") 

        Folder_name  Traning Image Numbers
0      glioma_tumor                    100
1  meningioma_tumor                    115
2          no_tumor                    105
3   pituitary_tumor                     74
--------------------------------------------------
items in x_test is:        394  items
items in y_test is:        394  items


In [9]:
#converting all TRAIN data to array
x_train = np.array(x_train) 
y_train = np.array(y_train)

x_valid = np.array(x_valid)
y_valid = np.array(y_valid)

x_test = np.array(x_test)
y_test = np.array(y_test)

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train,9)
y_valid = to_categorical(y_valid,9)
y_test = to_categorical(y_test,4)

print("x_train shape  :" ,x_train.shape)
print("y_train shape :", y_train.shape)
print('-----------------------------------------')
print("x_valid shape  :" ,x_valid.shape)
print("y_valid shape :", y_valid.shape)
print('-----------------------------------------')
print("x_test shape  :" ,x_test.shape)
print("y_test shape :", y_test.shape)


x_train shape  : (2296, 224, 224, 3)
y_train shape : (2296, 9)
-----------------------------------------
x_valid shape  : (574, 224, 224, 3)
y_valid shape : (574, 9)
-----------------------------------------
x_test shape  : (394, 224, 224, 3)
y_test shape : (394, 4)


In [10]:
#converting all TEST data to array
#shuffle data
from sklearn.utils import shuffle
x_train,y_train = shuffle(x_train,y_train)
x_valid,y_valid = shuffle(x_valid,y_valid)
x_test,y_test = shuffle(x_test,y_test)


In [11]:
#normalizing data
x_train=x_train/255.0
x_valid =x_valid/255.0
x_test =x_test/255.0

# building model

In [12]:
base_model = ResNet50(include_top=False, input_shape=(224, 224, 3), weights = 'imagenet')
# Taking the output of the last convolution block in ResNet50

# Flatten the output layer to 1 dimensionGlobalAveragePooling2D
x = Flatten()(base_model.output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(2048, activation='relu')(x)
# Adding a fully connected layer having en(image_class) neurons which will  give the probability of image 
predictions = Dense(len(image_class), activation='softmax')(x)

base_model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

# let's visualize layer names and layer indices to see the layers we will freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_1
1 conv1_pad
2 conv1_conv
3 conv1_bn
4 conv1_relu
5 pool1_pad
6 pool1_pool
7 conv2_block1_1_conv
8 conv2_block1_1_bn
9 conv2_block1_1_relu
10 conv2_block1_2_conv
11 conv2_block1_2_bn
12 conv2_block1_2_relu
13 conv2_block1_0_conv
14 conv2_block1_3_conv
15 conv2_block1_0_bn
16 conv2_block1_3_bn
17 conv2_block1_add
18 conv2_block1_out
19 conv2_block2_1_conv
20 conv2_block2_1_bn
21 conv2_block2_1_relu
22 conv2_block2_2_conv
23 conv2_block2_2_bn
24 conv2_block2_2_relu
25 conv2_block2_3_conv
26 conv2_block2_3_bn
27 conv2_block2_add
28 conv2_block2_out
29 conv2_block3_1_conv
30 conv2_block3_1_bn
31 conv2_block3_1_relu
32 conv2_block3_2_conv
33 conv2_block3_2_bn
34 conv2_block3_2_relu
35 conv2_block3_3_conv
36 conv2_block3_3_bn
37 conv2_block3_add
38 conv2_block3_out
39 conv3_block1_1_conv
40 conv3_block1_1_bn
41 conv3_block1_1_relu
42 conv3_block1_2_conv
43 conv3_block1_2_bn
44 conv3_block1_2_relu
45 conv3_block1_0_conv
46 conv3_block1_3_conv
47 conv3_block1_0_bn
48 conv3_block1_3_bn

In [13]:
base_model.summary()
        

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [14]:
train_layers=[176,171,168,165]

for i in train_layers:
    print('Layer name is:      ',base_model.layers[i].name)
    print('the input:   ',base_model.layers[i].input)
    print('the output:  ',base_model.layers[i].output)
    print('....................................................................')

Layer name is:       dense
the input:    KerasTensor(type_spec=TensorSpec(shape=(None, 100352), dtype=tf.float32, name=None), name='flatten/Reshape:0', description="created by layer 'flatten'")
the output:   KerasTensor(type_spec=TensorSpec(shape=(None, 2048), dtype=tf.float32, name=None), name='dense/Relu:0', description="created by layer 'dense'")
....................................................................
Layer name is:       conv5_block3_3_conv
the input:    KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 512), dtype=tf.float32, name=None), name='conv5_block3_2_relu/Relu:0', description="created by layer 'conv5_block3_2_relu'")
the output:   KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 2048), dtype=tf.float32, name=None), name='conv5_block3_3_conv/BiasAdd:0', description="created by layer 'conv5_block3_3_conv'")
....................................................................
Layer name is:       conv5_block3_2_conv
the input:    KerasTensor(type_spec=Tensor

In [15]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger, LearningRateScheduler,TensorBoard
def  training_opt(filename):
    
    file_dir='E:/4sd_t extension/python_code/brain/results/'
    filepath = os.path.join(file_dir,filename+'.hdf5')
    checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', save_best_only=True, verbose=1)    # model.save(filename, include_optimizer=False)
    earlystop=EarlyStopping( monitor="val_loss",  min_delta=0,   patience=10,  mode="auto")
    model_callbacks = [checkpoint, earlystop]
    return model_callbacks


In [ ]:
#optimize = SGD(learning_rate=0.0001, decay=0.9 / 4, momentum=0.9, nesterov=True)
optimize=Adam(learning_rate=0.001)
epochs=256
batch_size=64

for i in train_layers:
    Layername=base_model.layers[i].name
    print('fine_tuning layer name: -----------------------------    ', Layername)
    
    for layer in base_model.layers[i:-1]:
        layer.trainable = True
    
    DeTraC_model = Model(inputs=base_model.input, outputs=predictions)
    #DeTraC_model.compile(loss='categorical_crossentropy', optimizer=optimize, metrics=['accuracy']) 
    DeTraC_model.compile(loss='mean_squared_error', optimizer=optimize, metrics=['accuracy']) 
    history=DeTraC_model.fit(x_train,y_train, validation_data= (x_valid,y_valid), epochs=epochs, batch_size=batch_size, callbacks=training_opt(Layername), verbose=1)
    

fine_tuning layer name: -----------------------------     dense
Epoch 1/256
36/36 [==============================] - ETA: 0s - loss: 0.1793 - accuracy: 0.1799
Epoch 00001: val_accuracy improved from -inf to 0.18293, saving model to E:/4sd_t extension/python_code/brain/results\dense.hdf5


C:\Users\AL-FAJR\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


36/36 [==============================] - 245s 7s/step - loss: 0.1793 - accuracy: 0.1799 - val_loss: 0.1815 - val_accuracy: 0.1829
Epoch 2/256
35/36 [============================>.] - ETA: 4s - loss: 0.2017 - accuracy: 0.0915

In [ ]:
#showing results and model accuracy 
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']
#epochs=30
epochs_range = range(epochs)

plt.figure(figsize=(12, 6))
plt.grid()

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
from tensorflow.keras.models import load_model

model=load_model('E:/4sd_t extension/python_code/brain/results/DeTraC_model/dense.hdf5') #, compile = False
#model.evaluate(x_train,y_train)
#model.evaluate(x_valid,y_valid)

In [ ]:
model.summary()

In [ ]:
#https://newbedev.com/how-to-add-and-remove-new-layers-in-keras-after-loading-weights


predictions = Dense(4, activation='softmax',name='new_output')(model.layers[-2].output)
model2 = Model(inputs=model.inputs, outputs=predictions)
#model.compile(optimizer='adam', loss='categorical_crossentropy'
model2.summary()

In [ ]:
optimize = SGD(learning_rate=0.0001, decay=0.9 / 3, momentum=0.9, nesterov=True)
model2.compile(loss='categorical_crossentropy', optimizer=optimize, metrics=['accuracy']) 

model2.evaluate(x_test,y_test)

In [ ]:
y_test_pred = model2.predict(x_test)
y_prediction = np.argmax(y_test_pred, axis=1)

y_true = np.argmax(y_test, axis=1)

y_true.shape, y_prediction.shape

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(y_true, y_prediction)
cm



In [ ]:
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=cm ,  figsize=(5, 5))
plt.show()

In [ ]:
target = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

fig, ax = plot_confusion_matrix(conf_mat=cm,  figsize=(8, 8),
                                colorbar=False,
                                show_absolute=False,
                                show_normed=True,
                                class_names=target)
plt.show()

In [ ]:
from sklearn.metrics import classification_report

target_names =['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
print(classification_report(y_true, y_prediction, target_names=target_names))

In [ ]:
#https://stackoverflow.com/questions/68586561/sklearn-multi-class-problem-and-reporting-sensitivity-and-specificity
# compute sensitivity and specificity for multi classification python

from sklearn.metrics import precision_recall_fscore_support
res = []
for l in [0,1,2,3]:
    prec,recall,_,_ = precision_recall_fscore_support(np.array(y_true)==l,
                                                      np.array(y_prediction)==l,
                                                      pos_label=True,average=None)
    res.append([l,recall[0],recall[1]])
    
pd.DataFrame(res,columns = ['class','sensitivity','specificity'])

In [ ]:
y_test.shape, y_test_pred.shape

In [ ]:
# Compute ROC curve and ROC area for each class
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle

#pip install scikit-plot
target_names =['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

fpr = dict()
tpr = dict()
roc_auc = dict()
lw=2
n_classes=4

plt.figure(figsize=(9,9))
plt.grid()
plt.rcParams['font.size']=14

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_test_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
colors = cycle(['blue', 'red', 'green','orange'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='AUC for class {0} ={1:0.2f}' ''.format(target_names[i], roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize = 16)
plt.ylabel('True Positive Rate', fontsize = 16)
plt.title('ROC for brain tumor dataset', fontsize = 16)
plt.legend(loc="lower right", fontsize = 14)
plt.show()

In [ ]:
import dill
dill.dump_session('ResNet50_brain_env.db')

# DL-Reading_Data-2   تابع الملف عشان اعرف الباقي على نفس الطريقة الفوق

In [ ]:
# https://www.py4u.net/discuss/978360

%%html
<style type='text/css'>
.CodeMirror{
font-size: 17px;
</style>

In [ ]:
# we use SGD with a low learning rate

lr_schedule =tensorflow.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001,
    decay_steps=3,
    decay_rate=0.9)

opt = tensorflow.keras.optimizers.SGD(learning_rate=lr_schedule, , momentum=0.9) #name="SGD"

#compliling model
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])



In [ ]:
# plot and save confusion matrix
        fig, ax = plt.subplots(figsize=(16,12))
        plot_confusion_matrix(y_true, y_pred_class, ax=ax)
        fig.savefig(os.path.join(self.image_dir, f'confusion_matrix_epoch_{epoch}'))

       # plot and save roc curve
        fig, ax = plt.subplots(figsize=(16,12))
        plot_roc(y_true, y_pred, ax=ax)
        fig.savefig(os.path.join(self.image_dir, f'roc_curve_epoch_{epoch}'))